# Rancang Bangun Jaringan Saraf Tiruan untuk Sistem Kendali Lingkungan Termal Climate Chamber 

### Program Kecerdasan Buatan untuk membentuk model jaringan saraf tiruan dalam memenuhi Tugas Akhir program studi Teknik Fisika.

In [1]:
# Mengimpor pustaka untuk pengolahan data.
import numpy as np
import pandas as pd

from subprocess import check_output
print(check_output(["ls", "../Data"]).decode("utf8"))

data1Mar.xlsx
data2Jun.xlsx
data3Sep.xlsx
data4Des.xlsx
dataAll.xlsx
dataEDA.csv
DataFormat.xlsx
Data Simulasi IESVE (1-60).xlsx
Data Simulasi IESVE (61-80).xlsx
Data Simulasi IESVE (81-100).xlsx
Data Training ANN (1-60).xlsx
Data Training ANN (61-80).xlsx
Data Training ANN (81-100).xlsx
Data Training ANN Gabung (1-100).xlsx



In [2]:
# Mengimpor pustaka untuk visualisasi.
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns

In [3]:
# Memasukkan dataset ke dalam jupyter.
data = pd.read_excel('../Data/dataAll.xlsx')

In [4]:
# Menampilkan 5 data teratas.
data.head()

,No,Variation,Month,Time,Heater,AC,DrybulbT,Radiation,AirT,RH
0,1,DT001,Mar,00:03:00,0,0,24.08,0.0,24.03,70.64
1,2,DT001,Mar,00:09:00,0,0,24.08,0.0,24.03,70.72
2,3,DT001,Mar,00:15:00,0,0,24.08,0.0,24.04,70.81
3,4,DT001,Mar,00:21:00,0,0,24.08,0.0,24.04,70.90
4,5,DT001,Mar,00:27:00,0,0,24.08,0.0,24.04,70.99


In [5]:
# Membuang fitur No, Time dan DT.
data = data.drop(['No','Time'], axis =1)

In [6]:
# Memeriksa persebaran statistik data.
data.describe()

,Heater,AC,DrybulbT,Radiation,AirT,RH
count,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000
mean,0.600000,9.362500,25.269598,184.437794,24.528928,89.147813
std,0.800017,12.373357,1.825458,270.431525,2.489437,10.545183
min,0.000000,0.000000,13.130000,0.000000,16.000000,55.840000
25%,0.000000,0.000000,23.940000,0.000000,23.840000,80.680000
50%,0.000000,0.000000,24.680000,0.000000,24.660000,91.180000
75%,1.000000,24.000000,26.840000,291.576000,26.000000,99.870000
max,2.000000,30.000000,29.410000,845.891000,30.310000,100.000000


In [7]:
# Memeriksa nilai yang kosong.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 8 columns):
Variation    24000 non-null object
Month        24000 non-null object
Heater       24000 non-null int64
AC           24000 non-null int64
DrybulbT     24000 non-null float64
Radiation    24000 non-null float64
AirT         24000 non-null float64
RH           24000 non-null float64
dtypes: float64(4), int64(2), object(2)
memory usage: 1.5+ MB


## Klasifikasi menggunakan Multi-Layer Perceptron

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [9]:
# Memisahkan data input dan data target.
x = data[['DrybulbT','Radiation','AirT','RH']] # Data Input
y = data[['Heater','AC']]                      # Data Target

In [10]:
# Variabel untuk pembuatan grafik
DrybulbT  = x['DrybulbT']
Radiation = x['Radiation']
AirT      = x['AirT']
RH        = x['RH']
Heater    = y['Heater']
AC        = y['AC']

In [11]:
# One Hot Encoding
y = pd.get_dummies(y, columns = ['Heater','AC'])

In [12]:
# Memisahkan data pelatihan (training) dan data pengujian (testing).
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=(0.20), random_state=27)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(19200, 4) (4800, 4) (19200, 19) (4800, 19)


In [13]:
# Validasi data telah teracak
y_test.head()

,Heater_0,Heater_1,Heater_2,AC_0,AC_16,AC_17,AC_18,AC_19,AC_20,AC_21,AC_22,AC_23,AC_24,AC_25,AC_26,AC_27,AC_28,AC_29,AC_30
11033,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
18896,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6866,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
6012,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2093,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
x_train_v = x_train.values
x_test_v = x_test.values
y_train_v = y_train.values
y_test_v = y_test.values

In [15]:
# # Mencari jumlah hidden layer dan neuron optimal
# import operator

# def ann(m,n):
#     return MLPClassifier(hidden_layer_sizes=(m,n), max_iter=500, alpha=0.0001, solver='adam', verbose=10,  random_state=21, tol=0.000000001)

# daftar = {}
# for i in range(4,6):
#     for j in range(2,4):
#         model = ann(i,j)
#         model.fit(x_train_v, y_train_v)
#         y_pred = model.predict(x_test_v)
#         accuracy = accuracy_score(y_test_v, y_pred)
#         key = str(i) + ',' + str(j)
#         daftar[key] = accuracy
#         if accuracy >= 50:
#             neuron = {i,j}
#             break
#         else:
#             neuron = max(daftar.items(), key=operator.itemgetter(1))[0]

# print(neuron, daftar)

In [16]:
# Pembuatan model
# model_clf = MLPClassifier(activation='relu', hidden_layer_sizes=(1), max_iter=500, alpha=0.0001,
#                      solver='adam', verbose=10,  random_state=21, tol=0.000000001)
model_clf = MLPClassifier(random_state=1, activation='logistic', hidden_layer_sizes=(50))

In [17]:
model_clf.fit(x_train, y_train)     # Pelatihan model

/home/dhan/miniconda3/envs/ML/lib/python3.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
              beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=50, learning_rate='constant',
              learning_rate_init=0.001, max_iter=200, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [18]:
y_pred = model_clf.predict(x_test)  # Prediksi output
accuracy_score(y_test, y_pred)      # Menampilkan akurasi prediksi

0.24583333333333332

In [19]:
# y_pred = pd.DataFrame({"Heater":y_pred[:,0:3], "AC":y_pred[:,3:]})
# def get_animal(row):
#     for c in animals.columns:
#         if row[c]==1:
#             return c

In [20]:
# Menampilkan confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

ValueError: multilabel-indicator is not supported

# Visualisasi

In [ ]:
# plot a line, a perfit predict would all fall on this line
plt.subplot(2, 1, 1)
plt.title('Grafik AirT terhadap AC dan Heater')
plt.plot(AirT, AC, '.')
plt.plot(AirT, AC, 'r.')
plt.ylabel('AC')

plt.subplot(2, 1, 2)
plt.plot(AirT, Heater, '.')
plt.plot(AirT, Heater, 'r.')
plt.xlabel('Airt')
plt.ylabel('Heater')

plt.show()

In [ ]:
plt.subplot(2, 1, 1)
plt.title('Grafik AC terhadap AirT dan RH')
plt.plot(x_test.values[:,1], y_test.values[:,0], '.')
plt.plot(x_test.values[:,1], y_pred[:,0], 'r.')
plt.ylabel('AirT')

plt.subplot(2, 1, 2)
plt.plot(x_test.values[:,1], y_test.values[:,1], '.')
plt.plot(x_test.values[:,1], y_pred[:,1], 'r.')
plt.xlabel('AC')
plt.ylabel('RH')

plt.show()

In [ ]:
plt.subplot(2, 1, 1)
plt.title('Grafik DrybulbT terhadap AirT dan RH')
plt.plot(x_test.values[:,2], y_test.values[:,0], '.')
plt.plot(x_test.values[:,2], y_pred[:,0], 'r.')
plt.ylabel('AirT')

plt.subplot(2, 1, 2)
plt.plot(x_test.values[:,2], y_test.values[:,1], '.')
plt.plot(x_test.values[:,2], y_pred[:,1], 'r.')
plt.xlabel('DrybulbT')
plt.ylabel('RH')

plt.show()

In [ ]:
plt.subplot(2, 1, 1)
plt.title('Grafik Radiation terhadap AirT dan RH')
plt.plot(x_test.values[:,3], y_test.values[:,0], '.')
plt.plot(x_test.values[:,3], y_pred[:,0], 'r.')
plt.ylabel('AirT')

plt.subplot(2, 1, 2)
plt.plot(x_test.values[:,3], y_test.values[:,1], '.')
plt.plot(x_test.values[:,3], y_pred[:,1], 'r.')
plt.xlabel('Radiation')
plt.ylabel('RH')

plt.show()